- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Aplicar funcao de limpeza
- [ ] Salvar no banco

In [1]:
PASTA = './../../dados'

# Ler municipios

In [2]:
import duckdb
import pandas as pd

def ler_municipios(pasta_csvs):
    query = f"""
        SELECT DISTINCT estado, municipio
        FROM read_csv_auto('{pasta_csvs}/*.csv', union_by_name=true)
        WHERE estado IS NOT NULL AND municipio IS NOT NULL
        ORDER BY estado, municipio 
    """
    return duckdb.query(query).to_df()

In [ ]:
municipios_df = ler_municipios(PASTA)

len(municipios_df)

5570

# Ler os .csvs filtrando por municipio e estado

In [4]:
import duckdb
import pandas as pd

def ler_csvs_filtrando_municipio(pasta_csvs, municipio, estado, nome_coluna_municipio='municipio', nome_coluna_estado='estado'):
    municipio_escaped = municipio.replace("'", "''")
    estado_escaped = estado.replace("'", "''")
    
    query = f"""
        SELECT *
        FROM read_csv_auto('{pasta_csvs}/*.csv', union_by_name=true)
        WHERE {nome_coluna_municipio} = '{municipio_escaped}' AND {nome_coluna_estado} = '{estado_escaped}'
    """
    return duckdb.query(query).to_df()

# Funçao de limpeza

In [5]:
def suavizar(df, window_size=3, threshold=2):
    """
    Smooth data by replacing outliers with previous day's casosNovos.
    
    Parameters:
    - df: pandas DataFrame with columns 'date' and 'casosNovos'
    - window_size: number of days to consider in rolling window
    - threshold: number of standard deviations to use for outlier detection
    
    Returns:
    - DataFrame with smoothed values
    """
    # Make a copy to avoid modifying original data
    df_smoothed = df.copy()
    
    # Calculate rolling statistics
    rolling_mean = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).mean()
    rolling_std = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).std()
    
    # Identify outliers (values outside mean ± threshold*std)
    lower_bound = rolling_mean - threshold * rolling_std
    upper_bound = rolling_mean + threshold * rolling_std
    
    is_outlier = (df['casosNovos'] < lower_bound) | (df['casosNovos'] > upper_bound)
    
    # Replace outliers with previous day's value
    df_smoothed['casosNovos'] = df['casosNovos'].where(~is_outlier, df['casosNovos'].shift(1))
    
    # For the first row (no previous value), use the next value if available
    if is_outlier.iloc[0] and len(df) > 1:
        df_smoothed.iloc[0, df_smoothed.columns.get_loc('casosNovos')] = df['casosNovos'].iloc[1]
    
    return df_smoothed

# Recalcula casos acumulados

In [6]:
def recalcula_casos_acumulados(df):
  df['novos_casos_acumulados'] = df['casosNovos'].cumsum()
  return df

# Função de limpeza

In [7]:
def limpar(df):
  df = suavizar(df)
  df = recalcula_casos_acumulados(df)
  return df

# Função de logging

Funçao auxiliar

In [8]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler('logs.txt'),
        logging.StreamHandler()
    ]
)

# Função principal - sincrona e csv

In [ ]:
from tqdm import tqdm

OUTPUT_FOLDER = 'saida/'

sucesso = 'sucesso.txt'
falha = 'falha.txt'

for i in tqdm(range(len(municipios_df)), desc="Processando"):
  try:
    municipio = municipios_df.at[i, 'municipio']
    estado = municipios_df.at[i, 'estado']
    estado_municipio = f"{estado}_{municipio}"

    logging.info(f"Salvando {estado_municipio}")

    df = ler_csvs_filtrando_municipio(PASTA, municipio, estado)
    df = limpar(df)

    df.to_csv(f'saida/{estado_municipio}.csv', index=False)


    with open(sucesso, 'a', encoding='utf-8') as f:
      f.write(f"{estado_municipio}\n")

    logging.info(f"{estado}_{municipio} salvo")

  except Exception as e:
    with open(falha, 'a', encoding='utf-8') as f:
      f.write(f"{estado_municipio}\n")
    logging.error(f"{estado}_{municipio} não foi salvo")
    logging.error(e)

Processando:   0%|          | 0/5570 [00:00<?, ?it/s]2025-05-09 18:14:58 - INFO - Salvando AC_Acrelândia
2025-05-09 18:15:04 - INFO - AC_Acrelândia salvo
Processando:   0%|          | 1/5570 [00:05<8:55:39,  5.77s/it]2025-05-09 18:15:04 - INFO - Salvando AC_Assis Brasil
2025-05-09 18:15:09 - INFO - AC_Assis Brasil salvo
Processando:   0%|          | 2/5570 [00:11<8:36:38,  5.57s/it]2025-05-09 18:15:09 - INFO - Salvando AC_Brasiléia
2025-05-09 18:15:14 - INFO - AC_Brasiléia salvo
Processando:   0%|          | 3/5570 [00:16<8:10:52,  5.29s/it]2025-05-09 18:15:14 - INFO - Salvando AC_Bujari
2025-05-09 18:15:19 - INFO - AC_Bujari salvo
Processando:   0%|          | 4/5570 [00:21<8:05:37,  5.23s/it]2025-05-09 18:15:19 - INFO - Salvando AC_Capixaba
2025-05-09 18:15:25 - INFO - AC_Capixaba salvo
Processando:   0%|          | 5/5570 [00:26<8:09:28,  5.28s/it]2025-05-09 18:15:25 - INFO - Salvando AC_Cruzeiro do Sul
2025-05-09 18:15:30 - INFO - AC_Cruzeiro do Sul salvo
Processando:   0%|        